In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#!unzip "/content/drive/MyDrive/dataset/dataset/dataset.zip" -d "/content/drive/MyDrive"

In [6]:
import pathlib

import numpy as np
import tensorflow as tf

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB0
from tensorflow import keras
from keras.models import Model, load_model
import h5py



dataset_path = os.listdir('/content/drive/MyDrive/dataset')

# print (dataset_path)  #what kinds of classes are in this dataset
#
# print("Types of classes labels found: ", len(dataset_path))

class_labels = []

for item in dataset_path:
 # Get all the file names
 all_classes = os.listdir('/content/drive/MyDrive/dataset' + '/' +item)
 #print(all_classes)

 # Add them to the list
 for room in all_classes:
    class_labels.append((item, str('dataset_path' + '/' +item) + '/' + room))
    #print(class_labels[:5])

df = pd.DataFrame(data=class_labels, columns=['Labels', 'image'])
#print(df.head())
#print(df.tail())

# Let's check how many samples for each category are present
#print("Total number of images in the dataset: ", len(df))

label_count = df['Labels'].value_counts()
#print(label_count)


path = '/content/drive/MyDrive/dataset/'
#dataset_path = os.listdir('dataset')

im_size = 224

images = []
labels = []

for i in dataset_path:
    data_path = path + str(i)
    filenames = [i for i in os.listdir(data_path)]

    for f in filenames:
        img = cv2.imread(data_path + '/' + f)
        img = cv2.resize(img, (im_size, im_size))
        images.append(img)
        labels.append(i)

#This model takes input images of shape (224, 224, 3), and the input data should range [0, 255].

images = np.array(images)

images = images.astype('float32') / 255.0
images.shape

#preprocessing and Encoding
y=df['Labels'].values
#print(y)

y_labelencoder = LabelEncoder ()
y = y_labelencoder.fit_transform (y)
#print (y)

y=y.reshape(-1,1)

#ColumnTransformer
ct = ColumnTransformer([('my_ohe', OneHotEncoder(), [0])], remainder='passthrough')
Y = ct.fit_transform(y) #.toarray()
# print(Y[:5])
# print(Y[35:])

#Shuffle
#train_test_split


images, Y = shuffle(images, Y, random_state=1)


train_x, test_x, train_y, test_y = train_test_split(images, Y, test_size=0.05, random_state=415)

#inpect the shape of the training and testing.
# print(train_x.shape)
# print(train_y.shape)
# print(test_x.shape)
# print(test_y.shape)

#model
NUM_CLASSES = 2
IMG_SIZE = 224
size = (IMG_SIZE, IMG_SIZE)


inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))


# Using model without transfer learning

outputs = EfficientNetB0(include_top=True, weights=None, classes=NUM_CLASSES)(inputs)


#Model Fiting
model = tf.keras.Model(inputs, outputs)


model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"] )

# checkpoint_filepath = 'E:\codes\EfficentNetB0\Image-Classification-Using-EfficientNets\my_model'
#
# model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
#     filepath=checkpoint_filepath,
#     save_weights_only=True,
#     monitor='accuracy',
#     mode='max',
#     save_best_only=True)

model.summary()

checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model.fit(train_x, train_y, epochs=20, callbacks=[cp_callback], verbose=2)

model.save('n_model.h5')


#model.models.save_model("my_model.h5")
#model.save("my_model")
# tf.keras.models.save_model(
#     model,r"E:\codes\EfficentNetB0\Image-Classification-Using-EfficientNets\my_model", overwrite=True, include_optimizer=True, save_format='h5py',
#     signatures=None, options=None, save_traces=True
# )

# serialize model to JSON

# new_model =tf.keras.models.load_model('my_model.h5')
#
# loss, acc = new_model.evaluate(test_x, test_y)
# print("accuracy:{:5.2f}%".format(100*acc))








Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
efficientnetb0 (Functional)  (None, 2)                 4052133   
Total params: 4,052,133
Trainable params: 4,010,110
Non-trainable params: 42,023
_________________________________________________________________
Epoch 1/20
137/137 - 113s - loss: 1.8745 - accuracy: 0.5432

Epoch 00001: saving model to training_1/cp.ckpt
Epoch 2/20
137/137 - 70s - loss: 0.9383 - accuracy: 0.6490

Epoch 00002: saving model to training_1/cp.ckpt
Epoch 3/20
137/137 - 70s - loss: 0.8125 - accuracy: 0.7357

Epoch 00003: saving model to training_1/cp.ckpt
Epoch 4/20
137/137 - 70s - loss: 0.4479 - accuracy: 0.8435

Epoch 00004: saving model to training_1/cp.ckpt
Epoch 5/20
137/137 - 70s - loss: 0.3058 - accuracy: 0.9082

Epoch 00005: sa

/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [9]:
from matplotlib.pyplot import imread
from matplotlib.pyplot import imshow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.imagenet_utils import decode_predictions
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from tensorflow import keras
from keras.models import Model, load_model
import tensorflow as tf
import cv2
import numpy as np

loaded_model = load_model('/content/n_model.h5')


img_path = '/content/brain_tumor.jfif'

img = image.load_img(img_path, target_size=(224, 224))
#x = img.img_to_array(img)
x = np.array(img)

#img = cv2.imread(img_path)
#img = cv2.resize(img, (224, 224))

x = np.expand_dims(img, axis=0)
x = preprocess_input(x)

print('Input image shape:', x.shape)

my_image = imread(img_path)
#cv2.imshow(my_image)


preds= loaded_model.predict(x)
print(preds)

Input image shape: (1, 224, 224, 3)
[[1. 0.]]
